===================================================================================

- **Analisis Single ID Denpasar**
- *Data : CABANG AT_Denpasar*
- By **Zuhdi**
- Maret 2023

===================================================================================

In [1]:
# Library yang digunakan
import pandas as pd
import matplotlib as plt
pd.options.display.float_format = '{:,}'.format
float_frmt = lambda x: '{:,.0f}'.format(x) if x > 1e3 else '{:,.2f}'.format(x)
from datetime import datetime, date
import math
import numpy as np

# Data Sales

In [29]:
data_dps_sales = pd.read_excel('data_input/sales_singleid.xlsx',
                              sheet_name=0, index_col=False)
data_dps_sales.head(3)

,No,NO BP,TGL BP,NO SPK,TGL SPK,MODEL,TYPE MOBIL,NO MESIN,No Rangka,CARA BAYAR,...,FREE SERVICE,PIUTANG,ASURANSI.1,JENIS ASURANSI,TYPE BUYER,DURASI ASURANSI,ALL RISK (Tahun),TLO (Tahun),TGL DEC,T-CARE LITE+
0,1,205-MA150001,2015-01-06,205-IA20960,2015-01-05,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DEU5222,MHKM1CA4JEK086108,KREDIT,...,0,0,NaN,NaN,-,NaN,NaN,NaN,2015-02-26,0
1,2,205-MA150002,2015-01-06,205-IA20961,2015-01-05,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DEU6275,MHKM1CA4JEK086273,TUNAI,...,0,0,NaN,NaN,-,NaN,NaN,NaN,2015-02-26,0
2,3,205-MA150003,2015-01-06,205-IA20959,2015-01-02,INNOVA G MT BENSIN,TGN40R-GKMDKD A1,1TR 7942305,MHFXW42G0F2303957,KREDIT,...,0,0,NaN,NaN,-,NaN,NaN,NaN,2015-03-05,0


In [30]:
data_dps_sales.shape

(11055, 69)

## Treatment Missing Value

In [31]:
# Missing Value
# Fungsi untuk Cek missing Value

for col in data_dps_sales.columns:
    miss = data_dps_sales[col].isnull().sum()
    if miss>0:
        print("{} has {} missing value(s)".format(col,miss))
    else:
        print("{} has NO missing value!".format(col))

No has NO missing value!
NO BP has NO missing value!
TGL BP has NO missing value!
NO SPK has NO missing value!
TGL SPK has NO missing value!
MODEL has NO missing value!
TYPE MOBIL has NO missing value!
NO MESIN has 3 missing value(s)
No Rangka has NO missing value!
CARA BAYAR has NO missing value!
HARGA has NO missing value!
DISKON has NO missing value!
DISKON PROGRAM has NO missing value!
DISKON FLEET has NO missing value!
BBN has NO missing value!
CUSTOMER has 1 missing value(s)
VARIASI has NO missing value!
KOMISI has NO missing value!
DPP has NO missing value!
PPN has NO missing value!
NET SALES has NO missing value!
LEAD TIME has NO missing value!
LEASING has 6629 missing value(s)
LEASING OTHER has 10972 missing value(s)
SUBSIDI TAF has NO missing value!
TENOR has NO missing value!
RATE has NO missing value!
F/NF has 102 missing value(s)
NOPOL has 59 missing value(s)
NO BPKB has 522 missing value(s)
TGL BPKB has NO missing value!
SALES has 84 missing value(s)
ALAMAT1 has 2 missing

In [32]:
data_dps_sales.dropna(subset=['No Rangka','NIK'],inplace=True)

In [33]:
data_dps_sales.shape

(11055, 69)

## Treatment Duplikasi

In [27]:
data_dps_sales.drop_duplicates(subset=['No Rangka'],
                     keep=False, inplace=True)

In [28]:
data_dps_sales.shape

(11055, 69)

## Generate Single ID by NIK

In [34]:
data_dps_sales_nik = data_dps_sales.groupby(['NIK','F/NF']).agg({'No Rangka' : lambda unit : len(unit)})

In [35]:
# Ganti nama kolom 

data_dps_sales_nik.rename(columns={'No Rangka' : 'Jumlah Unit'} , inplace=True)

In [36]:
data_dps_sales_nik

,,Jumlah Unit
NIK,F/NF,
',Non Fleet,1
' 01.112.098.7-904.000,Non Fleet,2
' 01.777.235.1-904.000,Non Fleet,1
' 027/228/P2PBJ,Non Fleet,1
' 2212220028165,Non Fleet,1
...,...,...
'RA0008985,Non Fleet,1
'RE0080963,Non Fleet,1
'XDA014865,Non Fleet,1


In [37]:
#data_dps_sales_nik.to_excel('sales_singleid_nik_id.xlsx')

In [38]:
data_dps_sales_nik = pd.read_excel('data_input/sales_singleid_nik_id.xlsx',
                              sheet_name=0, index_col=False)

In [39]:
data_dps_sales_nik

,NIK,SINGLE ID
0,',DPS-00000001-RTL
1,' 01.112.098.7-904.000,DPS-00000002-RTL
2,' 01.777.235.1-904.000,DPS-00000003-RTL
3,' 027/228/P2PBJ,DPS-00000004-RTL
4,' 2212220028165,DPS-00000005-RTL
...,...,...
8660,'RA0008985,DPS-00008661-RTL
8661,'RE0080963,DPS-00008662-RTL
8662,'XDA014865,DPS-00008663-RTL
8663,'ZA985630,DPS-00008664-RTL


In [40]:
data_dps_sales_nik.shape

(8665, 2)

In [41]:
#Gabungan data NIK dengan Sales
data_dps_sales_nik_gab = pd.merge(data_dps_sales, data_dps_sales_nik, left_on='NIK', right_on='NIK', how='inner')

In [42]:
data_dps_sales_nik_gab

,No,NO BP,TGL BP,NO SPK,TGL SPK,MODEL,TYPE MOBIL,NO MESIN,No Rangka,CARA BAYAR,...,PIUTANG,ASURANSI.1,JENIS ASURANSI,TYPE BUYER,DURASI ASURANSI,ALL RISK (Tahun),TLO (Tahun),TGL DEC,T-CARE LITE+,SINGLE ID
0,2,205-MA150002,2015-01-06,205-IA20961,2015-01-05,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DEU6275,MHKM1CA4JEK086273,TUNAI,...,0,NaN,NaN,-,NaN,NaN,NaN,2015-02-26,0,DPS-00003050-RTL
1,3,205-MA150003,2015-01-06,205-IA20959,2015-01-02,INNOVA G MT BENSIN,TGN40R-GKMDKD A1,1TR 7942305,MHFXW42G0F2303957,KREDIT,...,0,NaN,NaN,-,NaN,NaN,NaN,2015-03-05,0,DPS-00007634-RTL
2,1899,205-MA160312,2016-03-30,205-IA23286,2016-03-28,INNOVA V AT BENSIN,TGN140R-MDTMKD 00,1TR A073661,MHFGW8EM7G1003459,KREDIT,...,0,NaN,NaN,-,NaN,NaN,NaN,2016-04-01,0,DPS-00007634-RTL
3,4,205-MA150004,2015-01-06,205-IA20965,2015-01-05,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DER8405,MHKM1CA4JEK081598,TUNAI,...,0,NaN,NaN,-,NaN,NaN,NaN,NaN,0,DPS-00004397-RTL
4,5,205-MA150005,2015-01-06,205-IA20969,2015-01-06,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DEU0542,MHKM1CA4JEK085156,TUNAI,...,0,NaN,NaN,-,NaN,NaN,NaN,2015-01-10,0,DPS-00007800-RTL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10968,11051,205-MA160578,2016-05-31,205-IA23551,2016-05-26,AVANZA 1.3 G A/T,F653RM-GQMFJ 00,1NR F123860,MHKM5EB3JGK005249,KREDIT,...,0,NaN,NaN,-,NaN,NaN,NaN,2016-06-06,0,DPS-00006611-RTL
10969,11052,205-MA160579,2016-05-31,205-IA23554,2016-05-28,HILUX_SC 4x2 M/T BENSIN,TGN110R-BTMXKD 00,1TR A039414,MR0EW8BB4G0202871,TUNAI,...,0,NaN,NaN,-,NaN,NaN,NaN,2016-06-30,0,DPS-00007026-RTL
10970,11053,205-MA160580,2016-05-31,205-IA23564,2016-05-30,INNOVA V AT BENSIN,TGN140R-MDTMKD 00,1TR A119363,MHFGW8EM1G1005918,KREDIT,...,0,NaN,NaN,-,NaN,NaN,NaN,2016-06-30,0,DPS-00004953-RTL
10971,11054,205-MA160581,2016-05-31,205-IA23559,2016-05-30,INNOVA G MT BENSIN,TGN140R-MDMSKD 00,1TR A123772,MHFJW8EM3G2310126,TUNAI,...,0,NaN,NaN,-,NaN,NaN,NaN,2016-06-15,0,DPS-00003044-RTL


## T Care GBSB Data

In [43]:
data_dps_tcare = pd.read_excel('data_input/tcare_singleid.xlsx',
                              sheet_name=0, index_col=False)
data_dps_tcare.head(3)

,No Rangka,Grouping
0,JTNGF3DH5L8026626,GBSB
1,JTNGF3DHXL8028257,GBSB
2,MR2KUAAG8L0001893,GBSB


In [46]:
#Gabungan data NIK dengan Sales
data_dps_sales_nik_tcare_gab = pd.merge(data_dps_sales_nik_gab, data_dps_tcare, left_on='No Rangka', right_on='No Rangka', how='left')

In [47]:
data_dps_sales_nik_tcare_gab.shape

(10973, 71)

In [48]:
data_dps_sales_nik_tcare_gab

,No,NO BP,TGL BP,NO SPK,TGL SPK,MODEL,TYPE MOBIL,NO MESIN,No Rangka,CARA BAYAR,...,ASURANSI.1,JENIS ASURANSI,TYPE BUYER,DURASI ASURANSI,ALL RISK (Tahun),TLO (Tahun),TGL DEC,T-CARE LITE+,SINGLE ID,Grouping
0,2,205-MA150002,2015-01-06,205-IA20961,2015-01-05,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DEU6275,MHKM1CA4JEK086273,TUNAI,...,NaN,NaN,-,NaN,NaN,NaN,2015-02-26,0,DPS-00003050-RTL,NaN
1,3,205-MA150003,2015-01-06,205-IA20959,2015-01-02,INNOVA G MT BENSIN,TGN40R-GKMDKD A1,1TR 7942305,MHFXW42G0F2303957,KREDIT,...,NaN,NaN,-,NaN,NaN,NaN,2015-03-05,0,DPS-00007634-RTL,NaN
2,1899,205-MA160312,2016-03-30,205-IA23286,2016-03-28,INNOVA V AT BENSIN,TGN140R-MDTMKD 00,1TR A073661,MHFGW8EM7G1003459,KREDIT,...,NaN,NaN,-,NaN,NaN,NaN,2016-04-01,0,DPS-00007634-RTL,NaN
3,4,205-MA150004,2015-01-06,205-IA20965,2015-01-05,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DER8405,MHKM1CA4JEK081598,TUNAI,...,NaN,NaN,-,NaN,NaN,NaN,NaN,0,DPS-00004397-RTL,NaN
4,5,205-MA150005,2015-01-06,205-IA20969,2015-01-06,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DEU0542,MHKM1CA4JEK085156,TUNAI,...,NaN,NaN,-,NaN,NaN,NaN,2015-01-10,0,DPS-00007800-RTL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10968,11051,205-MA160578,2016-05-31,205-IA23551,2016-05-26,AVANZA 1.3 G A/T,F653RM-GQMFJ 00,1NR F123860,MHKM5EB3JGK005249,KREDIT,...,NaN,NaN,-,NaN,NaN,NaN,2016-06-06,0,DPS-00006611-RTL,NaN
10969,11052,205-MA160579,2016-05-31,205-IA23554,2016-05-28,HILUX_SC 4x2 M/T BENSIN,TGN110R-BTMXKD 00,1TR A039414,MR0EW8BB4G0202871,TUNAI,...,NaN,NaN,-,NaN,NaN,NaN,2016-06-30,0,DPS-00007026-RTL,NaN
10970,11053,205-MA160580,2016-05-31,205-IA23564,2016-05-30,INNOVA V AT BENSIN,TGN140R-MDTMKD 00,1TR A119363,MHFGW8EM1G1005918,KREDIT,...,NaN,NaN,-,NaN,NaN,NaN,2016-06-30,0,DPS-00004953-RTL,NaN
10971,11054,205-MA160581,2016-05-31,205-IA23559,2016-05-30,INNOVA G MT BENSIN,TGN140R-MDMSKD 00,1TR A123772,MHFJW8EM3G2310126,TUNAI,...,NaN,NaN,-,NaN,NaN,NaN,2016-06-15,0,DPS-00003044-RTL,NaN


In [51]:
data_dps_sales_nik_tcare_gab['Grouping'].unique()

array([nan, 'GBSB', 'T-CARE', 'SA T-CARE', 'T-CARE Lite'], dtype=object)

In [52]:
data_dps_sales_nik_tcare_gab['Grouping'] = data_dps_sales_nik_tcare_gab['Grouping'].fillna('Regular')

In [53]:
data_dps_sales_nik_tcare_gab['Grouping'].unique()

array(['Regular', 'GBSB', 'T-CARE', 'SA T-CARE', 'T-CARE Lite'],
      dtype=object)

## Generate Data

In [54]:
#data_dps_sales_nik_tcare_gab.to_excel('sales_singleid_nik_final.xlsx')

# Data GS

In [197]:
data_dps_gs = pd.read_excel('data_input/gs_singleid.xlsx',
                              sheet_name=0, index_col=False)
data_dps_gs.head(3)

,No,No Invoice,Tgl,No Polisi,Customer,Labor,Part,Oli,Material,Sublet,...,Hpp Sublet,Status SI,kode Diskon,Kelurahan,Kecamatan,Last Service,Service Order,Tgl Service Order,CPUS,Unnamed: 48
0,1,205117000002,2017-01-01,DK-701-BF,ANITA IRAWATI,420900,0,0,5727,0,...,0,60,NaN,PEMOGAN,NaN,2017-11-19,205117000002,2017-01-01,0,NaN
1,2,205117000003,2017-01-01,DK-1106-BF,KARIYASA I MADE,747100,386365,987270,59363,0,...,0,60,NaN,PADANGSAMBIAN KAJA,NaN,2018-02-10,205117000003,2017-01-01,0,NaN
2,3,205117000004,2017-01-01,DK-1106-BF,KARIYASA I MADE,48200,0,0,95455,0,...,0,50,NaN,PADANGSAMBIAN KAJA,NaN,2018-02-10,205117000004,2017-01-01,0,NaN


In [198]:
data_dps_gs.shape

(184219, 49)

## Treatment Missing Value

In [199]:
# Missing Value
# Fungsi untuk Cek missing Value

for col in data_dps_gs.columns:
    miss = data_dps_gs[col].isnull().sum()
    if miss>0:
        print("{} has {} missing value(s)".format(col,miss))
    else:
        print("{} has NO missing value!".format(col))

No has NO missing value!
No Invoice has NO missing value!
Tgl has NO missing value!
No Polisi has 8 missing value(s)
Customer has 8 missing value(s)
Labor has NO missing value!
Part has NO missing value!
Oli has NO missing value!
Material has NO missing value!
Sublet has NO missing value!
Dpp has NO missing value!
PPN has NO missing value!
Meterai has NO missing value!
Total has NO missing value!
Faktur Pajak has 13904 missing value(s)
KM has NO missing value!
SBE has 11684 missing value(s)
Repair Type has 8 missing value(s)
Model has 768 missing value(s)
Diskon has NO missing value!
No Rangka has 631 missing value(s)
SA has 9 missing value(s)
Disc Jasa has NO missing value!
Disc Part has NO missing value!
Alamat has 164 missing value(s)
Contact Person has 2238 missing value(s)
TLP/HP has 164 missing value(s)
Delivery has NO missing value!
Flat Rate has NO missing value!
Stall Type has 283 missing value(s)
Jam Terpakai has NO missing value!
Cab.Penjual has 51597 missing value(s)
Sales 

In [30]:
#data_dps_gs.dropna(subset=['No Rangka', 'Customer','No Polisi','Alamat'],inplace=True)

In [31]:
#data_dps_gs.shape

(168231, 48)

## Treatment Duplikasi

In [58]:
data_dps_gs.drop_duplicates(subset=['No Rangka'],
                     keep=False, inplace=True)

In [59]:
data_dps_gs.shape

(11877, 49)

## RFM for GS

In [200]:
data_dps_gs['Tgl']= pd.to_datetime(data_dps_gs['Tgl'])

In [201]:
# Maximum Transaction Date or the latest transaction date.

max_trans_date = max(data_dps_gs['Tgl']).date()
max_trans_date

datetime.date(2023, 6, 30)

In [202]:
# Ambil tanggal data transaksi terakhir sebagai dasar perbandingan dan
# temukan jumlah hari antara transaction date dengan last transaksi date pada recency

comparison_date = datetime.strptime(str(max_trans_date), "%Y-%m-%d")

In [203]:
comparison_date

datetime.datetime(2023, 6, 30, 0, 0)

In [204]:
# Buatkan sebuah tabel RFM yang berisi nilai data recency(lama hari dari kedatangan) , frequency (Retensi) dan Monetary (Spend).

rfm_table_gs = data_dps_gs.groupby(['No Rangka']).agg({'Tgl': lambda date : (comparison_date - date.max()).days,
                        'No Rangka' : lambda retensi : len(retensi),
                        'Total' : lambda p : sum(p)})

In [205]:
rfm_table_gs

,Tgl,No Rangka,Total
No Rangka,,,
0,2299,1,533501
203449,273,1,9099247
906851,186,1,1656342
1510368,225,1,11121459
8181992,1467,1,3076250
...,...,...,...
ZSU600039792,18,2,805410
ZSU600042330,915,1,184140
ZSUMCU15600029410,1101,1,111980


In [206]:
# Cek Nama Kolom

rfm_table_gs.columns

Index(['Tgl', 'No Rangka', 'Total'], dtype='object')

In [207]:
# Ganti nama kolom sesuai kategori: recency, frequency, dan monetary

rfm_table_gs.rename(columns={'Tgl' : 'Days_Terakhir_Service', 
                        'No Rangka' : 'Retensi',
                        'Total' : 'Total Revenue'} , inplace=True)

In [208]:
rfm_table_gs

,Days_Terakhir_Service,Retensi,Total Revenue
No Rangka,,,
0,2299,1,533501
203449,273,1,9099247
906851,186,1,1656342
1510368,225,1,11121459
8181992,1467,1,3076250
...,...,...,...
ZSU600039792,18,2,805410
ZSU600042330,915,1,184140
ZSUMCU15600029410,1101,1,111980


In [209]:
# Menghitung AATS
rfm_table_gs['AATS'] = rfm_table_gs['Retensi']/4

In [210]:
# Menghitung Bulan Terakhir Service
rfm_table_gs['Bulan_Terakhir_Service'] = round(rfm_table_gs['Days_Terakhir_Service']/30,2)

In [211]:
rfm_table_gs['Average Revenue'] = round(rfm_table_gs['Total Revenue']/rfm_table_gs['Retensi'],2)

In [212]:
# RFM_table dataset

rfm_table_gs

,Days_Terakhir_Service,Retensi,Total Revenue,AATS,Bulan_Terakhir_Service,Average Revenue
No Rangka,,,,,,
0,2299,1,533501,0.25,76.63,"533,501.0"
203449,273,1,9099247,0.25,9.1,"9,099,247.0"
906851,186,1,1656342,0.25,6.2,"1,656,342.0"
1510368,225,1,11121459,0.25,7.5,"11,121,459.0"
8181992,1467,1,3076250,0.25,48.9,"3,076,250.0"
...,...,...,...,...,...,...
ZSU600039792,18,2,805410,0.5,0.6,"402,705.0"
ZSU600042330,915,1,184140,0.25,30.5,"184,140.0"
ZSUMCU15600029410,1101,1,111980,0.25,36.7,"111,980.0"


In [213]:
rfm_table_gs = rfm_table_gs.dropna()

In [214]:
rfm_table_gs

,Days_Terakhir_Service,Retensi,Total Revenue,AATS,Bulan_Terakhir_Service,Average Revenue
No Rangka,,,,,,
0,2299,1,533501,0.25,76.63,"533,501.0"
203449,273,1,9099247,0.25,9.1,"9,099,247.0"
906851,186,1,1656342,0.25,6.2,"1,656,342.0"
1510368,225,1,11121459,0.25,7.5,"11,121,459.0"
8181992,1467,1,3076250,0.25,48.9,"3,076,250.0"
...,...,...,...,...,...,...
ZSU600039792,18,2,805410,0.5,0.6,"402,705.0"
ZSU600042330,915,1,184140,0.25,30.5,"184,140.0"
ZSUMCU15600029410,1101,1,111980,0.25,36.7,"111,980.0"


In [215]:
rfm_table_gs.dtypes

Days_Terakhir_Service       int64
Retensi                     int64
Total Revenue               int64
AATS                      float64
Bulan_Terakhir_Service    float64
Average Revenue           float64
dtype: object

In [216]:
# menghilangkan angka tidak unik di f_quartil
def jitter(a_series, noise_reduction=1000000):
    return (np.random.random(len(a_series))*a_series.std()/noise_reduction)-(a_series.std()/(2*noise_reduction))

In [217]:
# Lakukan bining data
# Membagi recency, frekuensi, dan monetary menjadi 4 kuartil (min, 25%, 50%, 75% dan maks).
# Nilai-nilai ini akan membantu kita menghitung skor RFM untuk pelanggan dan mengklasifikasikannya berdasarkan skor RFM mereka.
# fungsi yang digunakan "qcut". yaitu proses bining data. Menurut dokumentasi pandas, qcut digambarkan sebagai Quantile-based discretization function. Singkatnya fungsi qcut() ini akan membagi data ke dalam jumlah yang sama. Karena itu, jarak untuk masing-masing bin boleh jadi berbeda satu sama lain

rfm_table_gs['r_quartile'] = pd.qcut(rfm_table_gs['Days_Terakhir_Service'], 4, ['4','3','2','1'])
rfm_table_gs['f_quartile'] = pd.qcut(rfm_table_gs['Retensi']+ jitter(rfm_table_gs['Retensi']), 4, ['1','2','3','4'])
rfm_table_gs['m_quartile'] = pd.qcut(rfm_table_gs['Total Revenue'], 4, ['1','2','3','4'])

In [218]:
rfm_table_gs

,Days_Terakhir_Service,Retensi,Total Revenue,AATS,Bulan_Terakhir_Service,Average Revenue,r_quartile,f_quartile,m_quartile
No Rangka,,,,,,,,,
0,2299,1,533501,0.25,76.63,"533,501.0",1,1,1
203449,273,1,9099247,0.25,9.1,"9,099,247.0",3,1,4
906851,186,1,1656342,0.25,6.2,"1,656,342.0",4,1,2
1510368,225,1,11121459,0.25,7.5,"11,121,459.0",3,1,4
8181992,1467,1,3076250,0.25,48.9,"3,076,250.0",2,1,3
...,...,...,...,...,...,...,...,...,...
ZSU600039792,18,2,805410,0.5,0.6,"402,705.0",4,2,2
ZSU600042330,915,1,184140,0.25,30.5,"184,140.0",2,1,1
ZSUMCU15600029410,1101,1,111980,0.25,36.7,"111,980.0",2,1,1


In [219]:
# Hitung Nilai RFM Score.
# Pembobotan maksimum diberikan untuk recency, kemudian frekuensi dan terakhir monetary

rfm_table_gs['rfm_score'] = 100*rfm_table_gs['r_quartile'].astype(int)+10*rfm_table_gs['f_quartile'].astype(int)+rfm_table_gs['m_quartile'].astype(int)

In [220]:
# Buatkan title customer.
# degan platinum adalah score tertinggi, dan dilanjutkan sampai ke score terendah yaitu bronze

rfm_table_gs['customer_title'] = pd.qcut(rfm_table_gs['rfm_score'], 4, ['Bronze','Silver','Gold','Platinum'])

In [221]:
rfm_table_gs

,Days_Terakhir_Service,Retensi,Total Revenue,AATS,Bulan_Terakhir_Service,Average Revenue,r_quartile,f_quartile,m_quartile,rfm_score,customer_title
No Rangka,,,,,,,,,,,
0,2299,1,533501,0.25,76.63,"533,501.0",1,1,1,111,Bronze
203449,273,1,9099247,0.25,9.1,"9,099,247.0",3,1,4,314,Gold
906851,186,1,1656342,0.25,6.2,"1,656,342.0",4,1,2,412,Platinum
1510368,225,1,11121459,0.25,7.5,"11,121,459.0",3,1,4,314,Gold
8181992,1467,1,3076250,0.25,48.9,"3,076,250.0",2,1,3,213,Silver
...,...,...,...,...,...,...,...,...,...,...,...
ZSU600039792,18,2,805410,0.5,0.6,"402,705.0",4,2,2,422,Platinum
ZSU600042330,915,1,184140,0.25,30.5,"184,140.0",2,1,1,211,Bronze
ZSUMCU15600029410,1101,1,111980,0.25,36.7,"111,980.0",2,1,1,211,Bronze


## Active, Passive, Sleeping dan Punctual, Non Punctual

In [222]:
def active_non(cols):
    
    active_score = cols[0]
    
    if active_score > 12:
        return 'Sleeping'
    elif active_score >6 and active_score <= 12:
        return 'Passive'
    else :
        return 'Active'

In [223]:
rfm_table_gs['Active/Non'] = rfm_table_gs[['Bulan_Terakhir_Service']].apply(active_non, axis=1)

In [224]:
rfm_table_gs['Active/Non'].unique()

array(['Sleeping', 'Passive', 'Active'], dtype=object)

In [225]:
rfm_table_gs

,Days_Terakhir_Service,Retensi,Total Revenue,AATS,Bulan_Terakhir_Service,Average Revenue,r_quartile,f_quartile,m_quartile,rfm_score,customer_title,Active/Non
No Rangka,,,,,,,,,,,,
0,2299,1,533501,0.25,76.63,"533,501.0",1,1,1,111,Bronze,Sleeping
203449,273,1,9099247,0.25,9.1,"9,099,247.0",3,1,4,314,Gold,Passive
906851,186,1,1656342,0.25,6.2,"1,656,342.0",4,1,2,412,Platinum,Passive
1510368,225,1,11121459,0.25,7.5,"11,121,459.0",3,1,4,314,Gold,Passive
8181992,1467,1,3076250,0.25,48.9,"3,076,250.0",2,1,3,213,Silver,Sleeping
...,...,...,...,...,...,...,...,...,...,...,...,...
ZSU600039792,18,2,805410,0.5,0.6,"402,705.0",4,2,2,422,Platinum,Active
ZSU600042330,915,1,184140,0.25,30.5,"184,140.0",2,1,1,211,Bronze,Sleeping
ZSUMCU15600029410,1101,1,111980,0.25,36.7,"111,980.0",2,1,1,211,Bronze,Sleeping


In [226]:
def punctual_non(cols):
    
    punctual_score = cols[0]
    
    if punctual_score > 6:
        return 'Non Punctual'
    else :
        return 'Punctual'

In [227]:
rfm_table_gs['Punctual/Non'] = rfm_table_gs[['Bulan_Terakhir_Service']].apply(punctual_non, axis=1)

In [228]:
rfm_table_gs['Punctual/Non'].unique()

array(['Non Punctual', 'Punctual'], dtype=object)

In [229]:
rfm_table_gs

,Days_Terakhir_Service,Retensi,Total Revenue,AATS,Bulan_Terakhir_Service,Average Revenue,r_quartile,f_quartile,m_quartile,rfm_score,customer_title,Active/Non,Punctual/Non
No Rangka,,,,,,,,,,,,,
0,2299,1,533501,0.25,76.63,"533,501.0",1,1,1,111,Bronze,Sleeping,Non Punctual
203449,273,1,9099247,0.25,9.1,"9,099,247.0",3,1,4,314,Gold,Passive,Non Punctual
906851,186,1,1656342,0.25,6.2,"1,656,342.0",4,1,2,412,Platinum,Passive,Non Punctual
1510368,225,1,11121459,0.25,7.5,"11,121,459.0",3,1,4,314,Gold,Passive,Non Punctual
8181992,1467,1,3076250,0.25,48.9,"3,076,250.0",2,1,3,213,Silver,Sleeping,Non Punctual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZSU600039792,18,2,805410,0.5,0.6,"402,705.0",4,2,2,422,Platinum,Active,Punctual
ZSU600042330,915,1,184140,0.25,30.5,"184,140.0",2,1,1,211,Bronze,Sleeping,Non Punctual
ZSUMCU15600029410,1101,1,111980,0.25,36.7,"111,980.0",2,1,1,211,Bronze,Sleeping,Non Punctual


## Prioritas Reminder

In [230]:
category_punctual1 = 'Punctual'
category_punctual2 = 'Non Punctual'
category_active1 = 'Active'
category_active2 = 'Passive'
category_active3 = 'Sleeping'

Punctual_Non  = rfm_table_gs['Punctual/Non']
Active_Non = rfm_table_gs['Active/Non']

In [231]:
# List of conditions
conditions = [
      (Punctual_Non == category_punctual1) & (Active_Non == category_active1)
    , (Punctual_Non == category_punctual1) & (Active_Non == category_active2)
    , (Punctual_Non == category_punctual1) & (Active_Non == category_active3)
    , (Punctual_Non == category_punctual2) & (Active_Non == category_active1)
    , (Punctual_Non == category_punctual2) & (Active_Non == category_active2)
    , (Punctual_Non == category_punctual2) & (Active_Non == category_active3)
]
# List of values to return
choices  = [
      1
    , 3
    , 5
    , 2
    , 4
    , 6
]
# create a new column in the DF based on the conditions
rfm_table_gs['Prioritas_Reminder'] = np.select(conditions, choices, "ERROR")

In [232]:
rfm_table_gs

,Days_Terakhir_Service,Retensi,Total Revenue,AATS,Bulan_Terakhir_Service,Average Revenue,r_quartile,f_quartile,m_quartile,rfm_score,customer_title,Active/Non,Punctual/Non,Prioritas_Reminder
No Rangka,,,,,,,,,,,,,,
0,2299,1,533501,0.25,76.63,"533,501.0",1,1,1,111,Bronze,Sleeping,Non Punctual,6
203449,273,1,9099247,0.25,9.1,"9,099,247.0",3,1,4,314,Gold,Passive,Non Punctual,4
906851,186,1,1656342,0.25,6.2,"1,656,342.0",4,1,2,412,Platinum,Passive,Non Punctual,4
1510368,225,1,11121459,0.25,7.5,"11,121,459.0",3,1,4,314,Gold,Passive,Non Punctual,4
8181992,1467,1,3076250,0.25,48.9,"3,076,250.0",2,1,3,213,Silver,Sleeping,Non Punctual,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZSU600039792,18,2,805410,0.5,0.6,"402,705.0",4,2,2,422,Platinum,Active,Punctual,1
ZSU600042330,915,1,184140,0.25,30.5,"184,140.0",2,1,1,211,Bronze,Sleeping,Non Punctual,6
ZSUMCU15600029410,1101,1,111980,0.25,36.7,"111,980.0",2,1,1,211,Bronze,Sleeping,Non Punctual,6


In [233]:
#Gabungan data RFM dengan GS
sales_gs_rfm = pd.merge(data_dps_gs, rfm_table_gs, left_on='No Rangka', right_on='No Rangka', how='inner')

In [234]:
sales_gs_rfm

,No,No Invoice,Tgl,No Polisi,Customer,Labor,Part,Oli,Material,Sublet,...,Bulan_Terakhir_Service,Average Revenue,r_quartile,f_quartile,m_quartile,rfm_score,customer_title,Active/Non,Punctual/Non,Prioritas_Reminder
0,1,205117000002,2017-01-01,DK-701-BF,ANITA IRAWATI,420900,0,0,5727,0,...,50.1,"621,941.2",2,3,3,233,Silver,Sleeping,Non Punctual,6
1,20963,205117021282,2017-08-27,DK-701-BF,ANITA IRAWATI,603900,140000,267726,97546,0,...,50.1,"621,941.2",2,3,3,233,Silver,Sleeping,Non Punctual,6
2,27602,205117027927,2017-11-19,DK-701-BF,ANITA IRAWATI,36600,622727,0,0,0,...,50.1,"621,941.2",2,3,3,233,Silver,Sleeping,Non Punctual,6
3,27616,205117027941,2017-11-20,DK-701-BF,ANITA IRAWATI,45750,0,0,0,0,...,50.1,"621,941.2",2,3,3,233,Silver,Sleeping,Non Punctual,6
4,75126,205119011544,2019-05-19,DK-1580-AM,ANITA IRAWATI,478400,32728,252273,133000,50000,...,50.1,"621,941.2",2,3,3,233,Silver,Sleeping,Non Punctual,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183583,184146,205123015193,2023-06-30,DK-863-IO,"SRI WAHYUNI, SE NI KETUT",43000,0,0,0,0,...,0.0,"47,730.0",4,1,1,411,Gold,Active,Punctual,1
183584,184148,205123015195,2023-06-30,DK-778-IG,MUSTIKA IDA BAGUS GEDE,43000,0,0,0,0,...,0.0,"47,730.0",4,1,1,411,Gold,Active,Punctual,1
183585,184164,205123015211,2023-06-30,DK-1650-BN,I NYOMAN KONDRA,735000,2216215,0,0,197684,...,0.0,"3,290,692.0",4,1,3,413,Platinum,Active,Punctual,1
183586,184176,205123015223,2023-06-30,B-2656-POW,PT. AGUNG SOLUSI TRANS,690000,91892,567570,0,0,...,0.0,"1,497,902.0",4,1,2,412,Platinum,Active,Punctual,1


In [235]:
sales_gs_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183588 entries, 0 to 183587
Data columns (total 63 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   No                      183588 non-null  int64         
 1   No Invoice              183588 non-null  int64         
 2   Tgl                     183588 non-null  datetime64[ns]
 3   No Polisi               183588 non-null  object        
 4   Customer                183588 non-null  object        
 5   Labor                   183588 non-null  int64         
 6   Part                    183588 non-null  int64         
 7   Oli                     183588 non-null  int64         
 8   Material                183588 non-null  int64         
 9   Sublet                  183588 non-null  int64         
 10  Dpp                     183588 non-null  int64         
 11  PPN                     183588 non-null  int64         
 12  Meterai                 183588

In [236]:
# Buatkan fungsi look up untuk menyesuaikan judul pelanggan berdasarkan skor RFM.

def title_skor_lkup(cols):
    
    rfm_score = cols[0]
    
    if rfm_score >= 444:
        return 'Platinum Customer'
    elif rfm_score >=433 and rfm_score < 444:
        return 'Very Loyal'
    elif rfm_score >=421 and rfm_score < 433:
        return 'Becoming Loyal'
    elif rfm_score >=344 and rfm_score < 421:
        return 'Recent Customer'
    elif rfm_score >=323 and rfm_score < 344:
        return 'Potential Customer'
    elif rfm_score >=311 and rfm_score < 323:
        return 'Late Bloomer'
    elif rfm_score >=224 and rfm_score < 311:
        return 'Loosing Customer'
    elif rfm_score >=212 and rfm_score < 224:
        return 'High Risk Customer'
    elif rfm_score >=124 and rfm_score < 212:
        return 'Almost Lost Customer'
    elif rfm_score >=112 and rfm_score < 124:
        return 'Evasive Customer'
    else :
        return 'Lost Customer'

In [237]:
# Menerapkan fungsi di atas dan membuat fitur baru detail_cust_title

sales_gs_rfm['title_customer']= sales_gs_rfm[['rfm_score']].apply(title_skor_lkup, axis=1)

In [238]:
# Fungsi untuk memberikan peringkat kepada pelanggan berdasarkan gelar mereka.

def get_rank(cols):
    
    title = cols[0]
    
    if title=='Platinum Customer':
        return 1
    elif title=='Very Loyal':
        return 2
    elif title == 'Becoming Loyal':
        return 3
    elif title == 'Recent Customer':
        return 4
    elif title=='Potential Customer':
        return 5
    elif title == 'Late Bloomer':
        return 6
    elif title == 'Loosing Customer':
        return 7
    elif title=='High Risk Customer':
        return 8
    elif title == 'Almost Lost Customer':
        return 9
    elif title == 'Evasive Customer':
        return 10
    else :
        return 11
        

In [239]:
# Menerapkan fungsi di atas dan membuat peringkat fitur baru

sales_gs_rfm['rank']=sales_gs_rfm[['title_customer']].apply(get_rank, axis=1)

In [240]:
sales_gs_rfm

,No,No Invoice,Tgl,No Polisi,Customer,Labor,Part,Oli,Material,Sublet,...,r_quartile,f_quartile,m_quartile,rfm_score,customer_title,Active/Non,Punctual/Non,Prioritas_Reminder,title_customer,rank
0,1,205117000002,2017-01-01,DK-701-BF,ANITA IRAWATI,420900,0,0,5727,0,...,2,3,3,233,Silver,Sleeping,Non Punctual,6,Loosing Customer,7
1,20963,205117021282,2017-08-27,DK-701-BF,ANITA IRAWATI,603900,140000,267726,97546,0,...,2,3,3,233,Silver,Sleeping,Non Punctual,6,Loosing Customer,7
2,27602,205117027927,2017-11-19,DK-701-BF,ANITA IRAWATI,36600,622727,0,0,0,...,2,3,3,233,Silver,Sleeping,Non Punctual,6,Loosing Customer,7
3,27616,205117027941,2017-11-20,DK-701-BF,ANITA IRAWATI,45750,0,0,0,0,...,2,3,3,233,Silver,Sleeping,Non Punctual,6,Loosing Customer,7
4,75126,205119011544,2019-05-19,DK-1580-AM,ANITA IRAWATI,478400,32728,252273,133000,50000,...,2,3,3,233,Silver,Sleeping,Non Punctual,6,Loosing Customer,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183583,184146,205123015193,2023-06-30,DK-863-IO,"SRI WAHYUNI, SE NI KETUT",43000,0,0,0,0,...,4,1,1,411,Gold,Active,Punctual,1,Recent Customer,4
183584,184148,205123015195,2023-06-30,DK-778-IG,MUSTIKA IDA BAGUS GEDE,43000,0,0,0,0,...,4,1,1,411,Gold,Active,Punctual,1,Recent Customer,4
183585,184164,205123015211,2023-06-30,DK-1650-BN,I NYOMAN KONDRA,735000,2216215,0,0,197684,...,4,1,3,413,Platinum,Active,Punctual,1,Recent Customer,4
183586,184176,205123015223,2023-06-30,B-2656-POW,PT. AGUNG SOLUSI TRANS,690000,91892,567570,0,0,...,4,1,2,412,Platinum,Active,Punctual,1,Recent Customer,4


In [241]:
#sales_gs_rfm.to_excel('gs_singleid_rfm.xlsx')

___

# Data BP

In [171]:
data_dps_bp = pd.read_excel('data_input/bp_singleid.xlsx',
                              sheet_name=0, index_col=False)
data_dps_bp.head(3)

C:\Users\ZUHDI AGUS\AppData\Local\Temp\ipykernel_11312\1401039843.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data_dps_bp = pd.read_excel('data_input/bp_singleid.xlsx',


,No,NOPOL,Customer,Service Order,Invoice No,Invoice Date,Customer Type,INSURANCE,SPK NUMBER,LABOR,...,NO HP,RECEIVE DATE,RECEIVE TIME,MODEL TYPE,VDS,DEALER,NOTES,MODEL KENDARAAN,No Rangka,CUSTOMER GROUP
0,1,39H0046554,ASURANSI MSIG INDONESIA PT. /,206918000015,"206,918,000,002.0",2018-01-02,ext,MSIG,-,836000,...,212524309,2018-01-02,15:58:00,PAS,8H39H,AAM,NaN,SIENTA,MHF8H39H0046554,perorangan
1,2,3JHK091551,ASURANSI MSIG INDONESIA PT. /,206918000014,"206,918,000,003.0",2018-01-02,ext,MSIG,-,770000,...,212524309,2018-01-02,15:50:00,COM,5EA3JH,AAM,NaN,AVANZA,MHK5EA3JH091551,perorangan
2,3,DK-702-LB,KETUT RAKA,206918000017,"206,918,000,004.0",2018-01-03,ext,RETAIL,RETAIL,583000,...,81338611256,2018-01-02,16:25:00,COM,FMRGK3,AAM,NaN,AVANZA,MHFFMRGK35K071295,perorangan


In [172]:
data_dps_bp.shape

(21254, 41)

## Treatment Missing Value

In [173]:
# Missing Value
# Fungsi untuk Cek missing Value

for col in data_dps_bp.columns:
    miss = data_dps_bp[col].isnull().sum()
    if miss>0:
        print("{} has {} missing value(s)".format(col,miss))
    else:
        print("{} has NO missing value!".format(col))

No has NO missing value!
NOPOL has NO missing value!
Customer has NO missing value!
Service Order has NO missing value!
Invoice No has 1 missing value(s)
Invoice Date has NO missing value!
Customer Type has NO missing value!
INSURANCE has NO missing value!
SPK NUMBER has NO missing value!
LABOR has NO missing value!
PART has NO missing value!
MATERIAL has NO missing value!
OIL has NO missing value!
SUBLET has NO missing value!
DISC JASA has NO missing value!
DISC PART has NO missing value!
DISC JASA(%) has NO missing value!
DISC PART(%) has NO missing value!
TOTAL DISCOUNT has NO missing value!
DP has NO missing value!
METERAI has NO missing value!
DPP has NO missing value!
OMZET NETT has NO missing value!
PPN has NO missing value!
TOTAL has NO missing value!
SERVICE ADVISOR has NO missing value!
TECHNICIAN has 22 missing value(s)
REPAIR DATE has 8 missing value(s)
REPAIR TYPE has NO missing value!
TINGKAT KESULITAN has 2 missing value(s)
NAMA CP has 76 missing value(s)
NO HP has 60 mi

In [174]:
data_dps_bp.dropna(subset=['No Rangka', 'Customer','NOPOL'],inplace=True)

In [175]:
data_dps_bp.shape

(21249, 41)

## Treatment Duplikasi

In [169]:
data_dps_bp.drop_duplicates(subset=['No Rangka'],
                     keep=False, inplace=True)

In [170]:
data_dps_bp.shape

(7226, 41)

## Generate Data

In [176]:
#data_dps_bp.to_excel('bp_singleid_final.xlsx')

____ 
*end of cleansing data*

# Data Gabungan

## Sales-GS

In [337]:
#load data sales_singleid
sales_singleid = pd.read_excel('data_input/sales_singleid_nik_final.xlsx',
                              sheet_name=0, index_col=False)
sales_singleid.head(3)

,NO BP,TGL BP,NO SPK,TGL SPK,MODEL,TYPE MOBIL,NO MESIN,No Rangka,CARA BAYAR,HARGA,...,ASURANSI.1,JENIS ASURANSI,TYPE BUYER,DURASI ASURANSI,ALL RISK (Tahun),TLO (Tahun),TGL DEC,T-CARE LITE+,SINGLE ID,Grouping
0,205-MA150002,2015-01-06,205-IA20961,2015-01-05,VELOZ VELOZ 1.5 M/T LUX,F652RM-GMSFJ 21,3SZ DEU6275,MHKM1CA4JEK086273,TUNAI,185800000,...,NaN,NaN,-,NaN,NaN,NaN,2015-02-26,0,DPS-00003050-RTL,Regular
1,205-MA150003,2015-01-06,205-IA20959,2015-01-02,INNOVA G MT BENSIN,TGN40R-GKMDKD A1,1TR 7942305,MHFXW42G0F2303957,KREDIT,277750000,...,NaN,NaN,-,NaN,NaN,NaN,2015-03-05,0,DPS-00007634-RTL,Regular
2,205-MA160312,2016-03-30,205-IA23286,2016-03-28,INNOVA V AT BENSIN,TGN140R-MDTMKD 00,1TR A073661,MHFGW8EM7G1003459,KREDIT,355950000,...,NaN,NaN,-,NaN,NaN,NaN,2016-04-01,0,DPS-00007634-RTL,Regular


In [338]:
#load data gs_singleid
gs_singleid = pd.read_excel('data_input/gs_singleid_rfm.xlsx',
                              sheet_name=0, index_col=False)
gs_singleid.head(3)

,No Invoice,Tgl,No Polisi,Customer,Labor,Part,Oli,Material,Sublet,Dpp,...,r_quartile,f_quartile,m_quartile,rfm_score,customer_title,Active/Non,Punctual/Non,Prioritas_Reminder,title_customer,rank
0,205117000002,2017-01-01,DK-701-BF,ANITA IRAWATI,420900,0,0,5727,0,426627,...,2,3,3,233,Silver,Sleeping,Non Punctual,6.0,Loosing Customer,7
1,205117021282,2017-08-27,DK-701-BF,ANITA IRAWATI,603900,140000,267726,97546,0,1109172,...,2,3,3,233,Silver,Sleeping,Non Punctual,6.0,Loosing Customer,7
2,205117027927,2017-11-19,DK-701-BF,ANITA IRAWATI,36600,622727,0,0,0,659327,...,2,3,3,233,Silver,NaN,NaN,NaN,Loosing Customer,7


### Left Join

In [339]:
sales_gs_singleid = pd.merge(left=gs_singleid, right=sales_singleid, how='left', left_on='No Rangka', right_on='No Rangka')

In [340]:
sales_gs_singleid

,No Invoice,Tgl,No Polisi,Customer,Labor,Part,Oli,Material,Sublet,Dpp,...,ASURANSI.1,JENIS ASURANSI,TYPE BUYER,DURASI ASURANSI,ALL RISK (Tahun),TLO (Tahun),TGL DEC,T-CARE LITE+,SINGLE ID,Grouping
0,205117000002,2017-01-01,DK-701-BF,ANITA IRAWATI,420900,0,0,5727,0,426627,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,205117021282,2017-08-27,DK-701-BF,ANITA IRAWATI,603900,140000,267726,97546,0,1109172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,205117027927,2017-11-19,DK-701-BF,ANITA IRAWATI,36600,622727,0,0,0,659327,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,205117027941,2017-11-20,DK-701-BF,ANITA IRAWATI,45750,0,0,0,0,45750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,205119011544,2019-05-19,DK-1580-AM,ANITA IRAWATI,478400,32728,252273,133000,50000,946401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183583,205123015193,2023-06-30,DK-863-IO,"SRI WAHYUNI, SE NI KETUT",43000,0,0,0,0,43000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183584,205123015195,2023-06-30,DK-778-IG,MUSTIKA IDA BAGUS GEDE,43000,0,0,0,0,43000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183585,205123015211,2023-06-30,DK-1650-BN,I NYOMAN KONDRA,735000,2216215,0,0,197684,3148899,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183586,205123015223,2023-06-30,B-2656-POW,PT. AGUNG SOLUSI TRANS,690000,91892,567570,0,0,1349462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Prioritas Call

In [341]:
category_prioritas1 = 1
category_prioritas2 = 2
category_prioritas3 = 3
category_prioritas4 = 4
category_prioritas5 = 5
category_prioritas6 = 6
category_group1 = ('T-CARE','SA T-CARE','SA T-CARE')
category_group4 = 'GBSB'
category_group5 = 'Regular'

Prioritas_Reminder  = sales_gs_singleid['Prioritas_Reminder']
Grouping = sales_gs_singleid['Grouping']

In [342]:
# List of conditions
conditions2 = [
      (Prioritas_Reminder == category_prioritas1) & (Grouping == category_group1)
    , (Prioritas_Reminder == category_prioritas1) & (Grouping == category_group4)
    , (Prioritas_Reminder == category_prioritas1) & (Grouping == category_group5)
    , (Prioritas_Reminder == category_prioritas2) & (Grouping == category_group1)
    , (Prioritas_Reminder == category_prioritas2) & (Grouping == category_group4)
    , (Prioritas_Reminder == category_prioritas2) & (Grouping == category_group5)
    , (Prioritas_Reminder == category_prioritas3) & (Grouping == category_group1)
    , (Prioritas_Reminder == category_prioritas3) & (Grouping == category_group4)
    , (Prioritas_Reminder == category_prioritas3) & (Grouping == category_group5)
    , (Prioritas_Reminder == category_prioritas4) & (Grouping == category_group1)
    , (Prioritas_Reminder == category_prioritas4) & (Grouping == category_group4)
    , (Prioritas_Reminder == category_prioritas4) & (Grouping == category_group5)
    , (Prioritas_Reminder == category_prioritas5) & (Grouping == category_group1)
    , (Prioritas_Reminder == category_prioritas5) & (Grouping == category_group4)
    , (Prioritas_Reminder == category_prioritas5) & (Grouping == category_group5)
    , (Prioritas_Reminder == category_prioritas6) & (Grouping == category_group1)
    , (Prioritas_Reminder == category_prioritas6) & (Grouping == category_group4)
    , (Prioritas_Reminder == category_prioritas6) & (Grouping == category_group5)
]
# List of values to return
choices2  = [
      '1'
    , '2b'
    , '3c'
    , '2a'
    , '3b'
    , '4c'
    , '3a'
    , '4b'
    , '5c'
    , '4a'
    , '5b'
    , '6c'
    , '5a'
    , '6b'
    , '7b'
    , '6a'
    , '7a'
    , '8'
]
# create a new column in the DF based on the conditions
sales_gs_singleid['Urutan_Prioritas_Call'] = np.select(conditions2, choices2, "Other")

In [343]:
sales_gs_singleid['Urutan_Prioritas_Call'].unique()

array(['Other', '8', '6c', '3c', '5b', '7a', '2b'], dtype=object)

In [344]:
sales_gs_singleid

,No Invoice,Tgl,No Polisi,Customer,Labor,Part,Oli,Material,Sublet,Dpp,...,JENIS ASURANSI,TYPE BUYER,DURASI ASURANSI,ALL RISK (Tahun),TLO (Tahun),TGL DEC,T-CARE LITE+,SINGLE ID,Grouping,Urutan_Prioritas_Call
0,205117000002,2017-01-01,DK-701-BF,ANITA IRAWATI,420900,0,0,5727,0,426627,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
1,205117021282,2017-08-27,DK-701-BF,ANITA IRAWATI,603900,140000,267726,97546,0,1109172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
2,205117027927,2017-11-19,DK-701-BF,ANITA IRAWATI,36600,622727,0,0,0,659327,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
3,205117027941,2017-11-20,DK-701-BF,ANITA IRAWATI,45750,0,0,0,0,45750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
4,205119011544,2019-05-19,DK-1580-AM,ANITA IRAWATI,478400,32728,252273,133000,50000,946401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183583,205123015193,2023-06-30,DK-863-IO,"SRI WAHYUNI, SE NI KETUT",43000,0,0,0,0,43000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
183584,205123015195,2023-06-30,DK-778-IG,MUSTIKA IDA BAGUS GEDE,43000,0,0,0,0,43000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
183585,205123015211,2023-06-30,DK-1650-BN,I NYOMAN KONDRA,735000,2216215,0,0,197684,3148899,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
183586,205123015223,2023-06-30,B-2656-POW,PT. AGUNG SOLUSI TRANS,690000,91892,567570,0,0,1349462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other


In [345]:
#sales_gs_singleid.to_excel('SingleID-DPS-Sales_GR_NIK.xlsx')

## Sales-BP

In [177]:
#load data gs_singleid
bp_singleid = pd.read_excel('data_input/bp_singleid_final.xlsx',
                              sheet_name=0, index_col=False)
bp_singleid.head(3)

,Unnamed: 0,No,NOPOL,Customer,Service Order,Invoice No,Invoice Date,Customer Type,INSURANCE,SPK NUMBER,...,NO HP,RECEIVE DATE,RECEIVE TIME,MODEL TYPE,VDS,DEALER,NOTES,MODEL KENDARAAN,No Rangka,CUSTOMER GROUP
0,0,1,39H0046554,ASURANSI MSIG INDONESIA PT. /,206918000015,"206,918,000,002.0",2018-01-02,ext,MSIG,-,...,212524309,2018-01-02,15:58:00,PAS,8H39H,AAM,NaN,SIENTA,MHF8H39H0046554,perorangan
1,1,2,3JHK091551,ASURANSI MSIG INDONESIA PT. /,206918000014,"206,918,000,003.0",2018-01-02,ext,MSIG,-,...,212524309,2018-01-02,15:50:00,COM,5EA3JH,AAM,NaN,AVANZA,MHK5EA3JH091551,perorangan
2,2,3,DK-702-LB,KETUT RAKA,206918000017,"206,918,000,004.0",2018-01-03,ext,RETAIL,RETAIL,...,81338611256,2018-01-02,16:25:00,COM,FMRGK3,AAM,NaN,AVANZA,MHFFMRGK35K071295,perorangan


### Left Join

In [179]:
sales_bp_singleid = pd.merge(left=bp_singleid, right=sales_singleid, how='left', left_on='No Rangka', right_on='No Rangka')

In [180]:
sales_bp_singleid

,Unnamed: 0,No,NOPOL_x,Customer,Service Order,Invoice No,Invoice Date,Customer Type,INSURANCE,SPK NUMBER,...,ASURANSI.1,JENIS ASURANSI,TYPE BUYER,DURASI ASURANSI,ALL RISK (Tahun),TLO (Tahun),TGL DEC,T-CARE LITE+,SINGLE ID,Grouping
0,0,1,39H0046554,ASURANSI MSIG INDONESIA PT. /,206918000015,"206,918,000,002.0",2018-01-02,ext,MSIG,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,3JHK091551,ASURANSI MSIG INDONESIA PT. /,206918000014,"206,918,000,003.0",2018-01-02,ext,MSIG,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,DK-702-LB,KETUT RAKA,206918000017,"206,918,000,004.0",2018-01-03,ext,RETAIL,RETAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,DK-1712-GM,NI WAYAN ERNAWATI,206918000006,"206,918,000,005.0",2018-01-03,ext,RETAIL,RETAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,DK-1054-BE,ASURANSI JASINDO PT. / NI NENGAH WERIA,206917004373,"206,918,000,006.0",2018-01-03,ext,JASINDO,ADJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21244,21249,21250,DK-1861-ACA,ASS RAMAYANA /ARUDI KURNIAWAN,205923001782,"205,923,001,728.0",2023-06-30,ext,RAMAYANA,7502122300228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21245,21250,21251,DK-1128-MP,ASS BCA /I MADE DAUH,205923001691,"205,923,001,729.0",2023-06-30,ext,BCA,109020223000424,...,NO ASURANSI,NaN,-,NaN,NaN,NaN,2023-02-02,0.0,DPS-00004159-RTL,T-CARE
21246,21251,21252,DK-1468-FBH,ASS ASTRA BUANA / I NYOMAN SUPARMA,205923001146,"205,923,001,730.0",2023-06-30,ext,AAB,3230032464,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21247,21252,21253,DK-1727-ACF,ASURANSI MEGA / WARTIKNO,205923001325,"205,923,001,731.0",2023-06-30,ext,MEGA,CR2305032365,...,NO ASURANSI,NaN,-,NaN,NaN,NaN,2023-03-31,0.0,DPS-00001353-RTL,Regular


In [186]:
#sales_bp_singleid.to_excel('testbp.xlsx')

## Durasi Asuransi

In [ ]:
sales_bp_singleid['']

## GR-BP

In [67]:
#load data SingleID-DPS-Sales_GR, data diambil dari data yang sudah di analisis Revenue, Retensi, AATS, dll
sales_gr_singleid = pd.read_excel('data_input/SingleID-DPS-Sales_GR_NIK.xlsx',
                              sheet_name=0, index_col=False)
sales_gr_singleid.head(3)

,Unnamed: 0,Unnamed: 0_x,No_x,No Invoice,Tgl,No Polisi,Customer,Labor,Part,Oli,...,NIP SALESMAN,NIP SUPERVISOR,DURASI ASURANSI,VIN,PEKERJAAN,SEGMEN BISNIS,SEGMEN LAINNYA,NAMA CABANG,CUSTOMER ID,SINGLE ID
0,0,0,7,205117000008,2017-01-01,DK-1395-FK,I KETUT MULIANA,36600,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,11,205117000012,2017-01-02,W-11-WI,SOEHOKO MOELJOTANTO,92000,581818,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,16,205117000017,2017-01-02,DK-1635-IY,ENDANG SAEPUL UBAD,698900,128181,847274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
sales_gr_singleid.shape

(41758, 150)

In [25]:
#load data SingleID-DPS-Sales_BP
sales_bp_singleid = pd.read_excel('data_input/SingleID-DPS-Sales_BP_NIK.xlsx',
                              sheet_name=0, index_col=False)
sales_bp_singleid.head(3)

,Unnamed: 0,Unnamed: 0_x,No_x,Police No,Customer,Service Order,Invoice No,Invoice Date,Customer Type,INSURANCE,...,NIP SALESMAN,NIP SUPERVISOR,DURASI ASURANSI,VIN,PEKERJAAN,SEGMEN BISNIS,SEGMEN LAINNYA,NAMA CABANG,CUSTOMER ID,SINGLE ID
0,0,0,1,DK-671-BA,ASS ASTRA BUANA / PT. PACTO,206917002588,206918000423,2018-02-08 00:00:00,ext,AAB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2,DK-1125-KL,ASS BINTANG PT. / NI KADEK PUTRI ADNYANI,206917003135,206918000331,2018-01-31 00:00:00,ext,BINTANG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,3,DK-1842-KY,ASS BINTANG ./ ANAK AGUNG GEDE,206917003309,206918000325,2018-01-31 00:00:00,ext,BINTANG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
sales_bp_singleid.shape

(13030, 130)

### Gabungkan

In [68]:
#pd.merge(left=sales_gr_singleid, right=sales_bp_singleid, how='left', left_on='No Rangka', right_on='No Rangka').to_excel('SingleID-DPS-GR_BP_NIK.xlsx')

___
*end of Gabungan Data*